In [ ]:
from psquery import twomassquery, psquery
from astropy import units as u
from astropy.coordinates import SkyCoord
import pylab as plt
import pandas as pd
import numpy as np
import glob, tqdm
from scipy import stats
%matplotlib inline

#### Read a csv file generated by running source extractor on the deep radio image. Here pyse is used:

example usage of pyse:

pyse  deep_image.fits --csv

In [ ]:
df = pd.read_csv('deep_image.csv')

In [ ]:
df['dec'] = df[' dec']

In [ ]:
df.drop(labels=[' dec'], axis=1)

In [ ]:
radius = 1 # arcsecond, Radius of query to the catalog 

In [ ]:
# for each source extracted from the radio image, query a catalogue and return nearby sources within a given radius 

sources = []
for i, row in tqdm.tqdm(df.iterrows()):
    # to query 2mass catalog, just use twomassquery.query_radec instead of psquery.query_radec here
    s = psquery.query_radec(ra=row.ra, dec=row.dec, radius=radius/3600)
    if s:
        if isinstance(s, list):
            src = SkyCoord(row.ra, row.dec, unit='deg')
            diff = []
            for l in s:
                diff.append(src.separation(l).to_value(u.arcsec))
            print(f'Separations are {diff}')
            sources.append(list(row) + [s[np.argmin(diff)]])
        else:
            sources.append(list(row) + [s])

In [ ]:
cross_matched_sources = pd.DataFrame(sources, columns=list(df.columns) + ['cat_pos'])

In [ ]:
print(f'Number of cross matched sources are: {len(cross_matched_sources)}')

In [ ]:
def ra_sep(x):
    s = SkyCoord(x.ra, x.dec, unit='deg')
    return (s.ra - x['cat_pos'].ra).to(u.arcsecond).value

def dec_sep(x):
    s = SkyCoord(x.ra, x.dec, unit='deg')
    return (s.dec - x['cat_pos'].dec).to(u.arcsecond).value

In [ ]:
cross_matched_sources['ra_sep'] = cross_matched_sources.apply(ra_sep, axis=1)
cross_matched_sources['dec_sep'] = cross_matched_sources.apply(dec_sep, axis=1)

In [ ]:
rar = np.array(cross_matched_sources['ra'])
decr = np.array(cross_matched_sources['dec'])

In [ ]:
rad = cross_matched_sources['ra_sep']
decd = cross_matched_sources['dec_sep']

In [ ]:
cos_dec = np.cos(np.radians(float(np.median(decr))))

In [ ]:
ra_off = np.median(rad)
dec_off = np.median(decd)
e_raoff = stats.median_absolute_deviation(rad)/0.6744  
e_decoff = stats.median_absolute_deviation(decd)/0.6744  

In [ ]:
# printing the offsets in RA and DEC along with the error on the estimated offset (all in arcseconds)
print('{0:1f} +- {1:1f}, {2:1f} +- {3:1f}'.format(ra_off*cos_dec, 
                                                  e_raoff*cos_dec, 
                                                  dec_off, e_decoff))

In [ ]:
plt.hist(cross_matched_sources['ra_sep']*cos_dec)
plt.xlabel('RA Offset (arcsecond)')

In [ ]:
plt.hist(cross_matched_sources['dec_sep'])
plt.xlabel('Dec Offset (arcsecond)')